# Classifying time series

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tommoral/24-sacl-ai-4-sciences/blob/main/session-3/02-classifying-signals.ipynb)

Authors: Thomas Moreau

<div align='center'>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Cloaked_in_red.jpg/520px-Cloaked_in_red.jpg" width="400" style="display:inline-block;"/> <span minimum-width=150 style="display:inline-block;"></span><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Pluto-Charon_system-new.gif/440px-Pluto-Charon_system-new.gif" width="350" style="display:inline-block;"/>
</div>

## Variable stars

Most stars emit light steadily in time, but a small fraction of them has a variable light curve: light emission versus time. We call them variable stars. The light curves are usually periodic and highly regular. There are essentially two reasons why light emission can vary. First, the star itself can be oscillating, so its light emission varies in time. Second, the star that seems a single point at Earth (because of our large distance) is actually a binary system: two stars that orbit around their common center of gravity. When the orbital plane is parallel to our line of view, the stars eclipse each other periodically, creating a light curve with a charateristic signature. Identifying, classifying, and analyzing variable stars are hugely important for calibrating distances, and making these analyses automatic will be crucial in the upcoming sky survey projects such as LSST.

In this notebook, we consider an extract of 2912 stars from the [EROS-1 catalog](http://eros.in2p3.fr/) with 4 types of variable light curves:
- [Eclipsing binary](http://en.wikipedia.org/wiki/Binary_star#Eclipsing_binaries),
- [Cepheid](http://en.wikipedia.org/wiki/Cepheid_variable),
- [RR-Lyrae](http://en.wikipedia.org/wiki/RR_Lyrae_variable),
- [Mira](http://en.wikipedia.org/wiki/Mira_variable).

Our goal is to learn a function that assign an observed variable star to one of these 4 classes. The observations are composed of the light curves in two frequency bands (red and blue), as well as static variables. 

Let us for load the data:

In [ ]:
from pathlib import Path
import pandas as pd

DATA_FILE = "data/variable_stars.parquet"
URL_REPO = "https://github.com/x-datascience-datacamp/datacamp-master/raw/main/session-3/"

if Path(DATA_FILE).exists():
    X_df = pd.read_parquet(DATA_FILE)
else:
    X_df = pd.read_parquet(f"{URL_REPO}{DATA_FILE}")

y = X_df['type']
X_df = X_df.drop(columns='type')


In [ ]:
X_df.head()

There are a few informative static variables in this dataframe:

<ul>
    <li> <code><b><span style="color:black">period</span></b></code>: the estimated period of the light curve.
    <li> <code><b>magnitude_b, magnitude_r</b></code>: The average apparent luminosity of the star (in two frequency bands). <a href="http://www.astro-tom.com/technical_data/magnitude_scale.htm">Magnitude</a> is a logarithmic measure, and the higher it is, the lower the apparent luminosity is.
    <li> <code><b><span style="color:black">log_p_not_variable</span></b></code>: Logarithm of the estimated probability that the star is stable.
    <li> <code><b><span style="color:black">sigma_flux_b, sigma_flux_r</span></b></code>: The square root of the total variance of the light measurements (indicating the amplitude of the variability).
    <li> <code><b><span style="color:red">type</span></b></code>: The label to predict.
</ul> 

Let us first start with a simple model based on these static features.
We start by splitting the data between train and test:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=20240707)

Now we will start with a dummy method, to see how hard this problem is:

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

clf = DummyClassifier(
    strategy="most_frequent"
).fit(X_train, y_train)

print("Accuracy:", accuracy_score(y_test, clf.predict(X_test)))
print("Balanced accuracy:", balanced_accuracy_score(y_test, clf.predict(X_test)))

<div class="alert alert-success">
    <b>EXERCISE</b>:
    Using what you learn yesterday, can you create a pipeline with the following steps
     <ul>
         <li>A column transformer that selects columns <code>'magnitude_b', 'magnitude_r', 'sigma_flux_b', 'sigma_flux_r', 'log_p_not_variable'</code>,</li>
         <li>A <code>RandomForestClassifier</code></li>
    </ul>
    
 <i>Hint:</i> To select the columns without modifying them, you can use <code>'passthrough'</code> as a <code>Transformer</code> in the <code>ColumnTransformer</code>.
</div>

Solution: `solutions/02-1_static-pipeline.py`


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier

model = make_pipeline(
    ...  # to fill
)
model.fit(X_train, y_train)
print("Accuracy:", accuracy_score(y_test, model.predict(X_test)))
print("Balanced accuracy:", balanced_accuracy_score(y_test, model.predict(X_test)))

### Constructing a classifier based on the light curves

We will now turn toward building a classifier based on the raw light curves, that are stored in `time_points_r/b` and `light_points_r/b` columns of our dataset.

A first step when working with signals is usually to display them and take a look at their properties.

In [ ]:
import matplotlib.pyplot as plt

for idx in [10, 11, 12]:
    star = X_df.iloc[idx]
    print(f"Number of time points for star {idx}: {len(star['time_points_b'])}")

    plt.scatter(star['time_points_b'], star['light_points_b'])

<div class="alert alert-info">
    <b>QUESTION</b>:
     <ul>
         <li>What can you say about these time-series?
             <i>Are they regular? Comparable? Noisy?</i></li>
         <li>How would you compare them?</li>
    </ul>
</div>


### Aligning and resampling the signals

The benefit of working with global features is that they can often be computed even with unaligned time-series.
However, these features can fail to catpure some interesting notion of similarity between the samples.

We will now explore how to align and resample the data.

Here, we have data that is periodic, and we are informed about the period by the column `period`.

In [ ]:
X_df['period']

We can thus look at the data on one period:

In [ ]:
import matplotlib.pyplot as plt

for idx in [10, 11, 100]:
    star = X_df.iloc[idx]
    print(f"Number of time points for star {idx}: {len(star['time_points_b'])}")

    plt.scatter(star['time_points_b'] % star['period'], star['light_points_b'])

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>How would you modify this code to "align" the time series?</li>
         <li>How would you modify this code to cope with the fact that the sample are not aligned?</li>
    </ul>
    
   *Hint:* resampling non-uniform points can be performed with the [`scipy.interpolate.interp1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) function.
</div>

Solution: `solutions/02-2_align_periodic_signals.py`

We can now do a `scikit-learn` pipeline directly on the resampled light curves using the `FunctionTransformer`.
`FunctionTransformer` allow you to use any kind of function transforming your data in a `scikit-learn` pipeline.
The caveats of this tranformer is that it cannot do anything during the `fit` part of the pipeline.
To create a new `Transformer` that resample the data, you can use:

In [ ]:
import numpy as np
from scipy import interpolate
from sklearn.preprocessing import FunctionTransformer

t = np.linspace(0, 1, 100)
def resample(star):
    t_i = (star['time_points_b'] % star['period']) / star['period']
    interp = interpolate.interp1d(
        t_i, star['light_points_b'],
        kind='linear', bounds_error=False, fill_value="extrapolate"
    )
    return pd.Series({k: v for k, v in zip(t, interp(t))})

X_train_t = FunctionTransformer(lambda X: X.apply(resample, axis=1)).transform(X_train)
X_train_t

<div class="alert alert-success">
    <b>EXERCISE</b>:
    Extend your static pipeline with the resampled time-series.
</div>

Solution: `solutions/02-3_static-pipeline.py`

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier

model = make_pipeline(
    ...  # to fill
)
model.fit(X_train, y_train)
print("Accuracy:", accuracy_score(y_test, model.predict(X_test)))
print("Balanced accuracy:", balanced_accuracy_score(y_test, model.predict(X_test)))

### Using a time-series classifiers with `tslearn` and `pyts`

We will now cover how to use classifiers adapted to the time-series, using the [`tslearn`](https://tslearn.readthedocs.io/en/stable/gen_modules/neighbors/tslearn.neighbors.KNeighborsTimeSeriesClassifier.html) and [`pyts`](https://pyts.readthedocs.io/en/stable/generated/pyts.classification.TimeSeriesForest.html#pyts.classification.TimeSeriesForest) packages.

In [ ]:
# !pip install tslearn pyts

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

clf = make_pipeline(
    FunctionTransformer(lambda X: X.apply(resample, axis=1)),
    KNeighborsTimeSeriesClassifier(n_neighbors=5, metric="dtw", n_jobs=-1)
).fit(X_train[:100], y_train[:100])

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from tslearn.svm import TimeSeriesSVC

clf = make_pipeline(
    FunctionTransformer(lambda X: X.apply(resample, axis=1)),
    TimeSeriesSVC()
).fit(X_train[:100], y_train[:100])

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from pyts.classification import TimeSeriesForest

clf = make_pipeline(
    FunctionTransformer(lambda X: X.apply(resample, axis=1)),
    TimeSeriesForest(n_windows=5)
).fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_test, y_pred))

Note that these two package implements many algorithms, even for unsupervised learning.
They all expect the signals to have the same length.



### Extracting global features with `tsfresh`


`Tsfresh` is a package aimed at automatically extracting global features from time series.
It expects as input `pandas.DataFrame`, with a `id` column that indicates to which the signal each row is linked, and a `time` column that indicate the time the sample has been observed.

In [ ]:
#!pip install -U tsfresh aeon

With this format, it is convenient to represent non-uniform time-series, with variable lenghts.
Let us create these `DataFrame` for the train and test:

In [ ]:
df_ts_train_b = pd.DataFrame([
    {'id': idx, 'time': t, 'light_b': v}
    for idx, row in X_train.iterrows()
    for t, v in zip(row['time_points_b'], row['light_points_b'])
])

df_ts_test_b = pd.DataFrame([
    {'id': idx, 'time': t, 'light_b': v}
    for idx, row in X_test.iterrows()
    for t, v in zip(row['time_points_b'], row['light_points_b'])
])
df_ts_train_b

`Tsfresh` provides a convenient helper to extract many possible features at once.

In [ ]:
from tsfresh import extract_relevant_features
extracted_features = extract_relevant_features(df_ts_train_b, y_train, column_id="id", column_sort="time")
extracted_features

In [ ]:
extracted_features.shape

They provide a convenient `scikit-learn` like API, that allow working with pipelines:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier

from aeon.transformations.collection.feature_based import TSFresh

from sklearn import set_config
set_config(transform_output='pandas')

clf = make_pipeline(
    make_column_transformer(
        ('passthrough', ['magnitude_b', 'magnitude_r', 'sigma_flux_b', 'sigma_flux_r', 'log_p_not_variable']),
        (make_pipeline(
            FunctionTransformer(lambda X: X.apply(resample, axis=1)),
            TSFresh()
        ), ['time_points_b', 'light_points_b', 'period'])
    ),
    RandomForestClassifier()
).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_test, y_pred))


## Clustering with `tslearn`

The `tslearn` library also provides algorithms to do some clustering with the time series using adapted distances.

In [ ]:
from sklearn.pipeline import  make_pipeline, FunctionTransformer
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans


km = make_pipeline(
    FunctionTransformer(lambda X: X.apply(resample, axis=1)),
    TimeSeriesScalerMeanVariance(),
    TimeSeriesKMeans(
        n_clusters=5, metric="softdtw", max_iter=5,
        max_iter_barycenter=5, random_state=0
    )
)
y_pred = km.fit_predict(X_train[:100])

In [ ]:
plt.figure(figsize=(12, 6))
for yi in range(5):
    plt.subplot(3, 3, yi + 1)
    X_t = km[:-1].transform(X_train[:100])
    for xx in X_t[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(km[-1].cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, 100)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Euclidean $k$-means")